<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import os
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import ipysheet
import colorsys
import xlsxwriter
from tqdm import tqdm_notebook as tqdm
from datetime import datetime

In [2]:
def to_groups(peds, interval, freq):
    grouped = peds.groupby(pd.Grouper(key='datetime', freq=str(interval) + freq))
    grouped = list(map(lambda x: x[1], list(grouped)))
    
    return grouped


def to_pedestrians(peds, mapping, group):
    df = peds.loc[peds['timestamp'].isin(group.timestamp.values)]
    pedestrians = mapping.loc[mapping['pedestrianId'].isin(df.pedestrianId.unique())]
    
    return pedestrians


def create_od_matrix(pedestrians, axis, precision=8, cap=None):
    total = len(pedestrians.pedestrianId.values)
    grouped = pedestrians.groupby(['source','target']).size().reset_index().rename(columns={0:'count'})
    grouped['count'] = grouped['count'] / total * 100
    
    od = np.zeros((len(axis), len(axis)))

    for i, s in enumerate(axis):
        for j, t in enumerate(axis):
            row = grouped.loc[(grouped['source'] == s) & (grouped['target'] == t)]
            value = 0
            
            if not row.empty:
                value = row['count'].values[0]
            
            if cap is not None and value < cap:
                value = 0
            
            od[i, j] = round(value, precision)
            
    return od


def generate_gradient_function(max_val):
    return """function (value) {
            if (value == 0) {
                return {}
            }
            var color1 = {red: 255, green: 0, blue: 0};
            var color2 = {red: 255, green: 255, blue: 0};
            var color3 = {red: 0, green: 255, blue: 0};
        
            var percent = value/100;
            
            if (percent <= 0.5) {
                start = color1;
                end = color2;
                percent = percent / 0.5
            } else {
                start = color2;
                end = color3;
                percent = (percent - 0.5) / 0.5
            }
        
            start = color1;
            end = color3;
        
            var diffRed = end.red - start.red;
            var diffGreen = end.green - start.green;
            var diffBlue = end.blue - start.blue;

            var g = {
              r: start.red + (diffRed * percent),
              g: start.green + (diffGreen * percent),
              b: start.blue + (diffBlue * percent),
            };
      
            return {
                backgroundColor: 'rgba('+g.r+','+g.g+','+g.b+', 0.2)'
            };
        }
        """

In [3]:
def run(peds, mapping, cap=None, precision=8, prefix=''):
    to_string = np.vectorize(lambda x: str(x))
    to_precision = np.vectorize(lambda x: round(x, precision))
    
    freq = 'T'
    interval = 10
    group = 0
    
    if 'datetime' not in peds.columns:
        peds.insert(1, 'datetime', pd.to_datetime(peds['timestamp'], unit='ms'))
    
    wFreq = widgets.Dropdown(options=[('hour', 'H'), ('min', 'T'), ('seconds', 'S')], value=freq, description='freq')
    wInterval = widgets.IntSlider(min=1, max=60, step=1, value=interval, description='interval', continuous_update=False)
    wGroup = widgets.IntSlider(min=0, max=0, step=1, value=group, description='group', continuous_update=False)
    wTime = widgets.Label(value="text")
    wSave = widgets.Button(description='Save all')
    
    
    source_target = np.union1d(mapping.source.unique(), mapping.target.unique())
    dim = len(source_target)
    axis = list(to_string(source_target)) + ['sum']
    sheet = ipysheet.sheet(rows=len(axis), columns=len(axis), column_headers=axis, row_headers=axis)
    
    groups = to_groups(peds, interval, freq)
    wGroup.max = len(groups) - 1
    
    start = groups[group].datetime.iloc[0]
    end = groups[group].datetime.iloc[-1]
    wTime.value = "{:%d, %b %Y} from {:%I:%M:%S %p} to {:%I:%M:%S %p}".format(start, start, end)
    
    pedestrians = to_pedestrians(peds, mapping, groups[group])
    od = create_od_matrix(pedestrians, source_target, precision=precision, cap=cap)
    
    origin = to_precision(od.sum(axis=0))
    destination = to_precision(od.sum(axis=1))
    
    ipysheet.renderer(code=generate_gradient_function(od.max()), name='gradient');
    ipysheet.renderer(code=generate_gradient_function(origin.max()), name='origin');
    ipysheet.renderer(code=generate_gradient_function(destination.max()), name='destination');
    
    ipysheet.cell_range(od, row_start=0, column_start=0, renderer='gradient')
    ipysheet.row(dim, origin, column_end=dim-1, renderer='origin')
    ipysheet.column(dim, destination, row_end=dim-1, renderer='destination')
    
    def f(change):
        nonlocal peds
        nonlocal mapping
        nonlocal freq
        nonlocal interval
        nonlocal group
        nonlocal groups
        nonlocal wGroup
        
        name = change['owner'].description
        value = change['new']
        
        regroup = False
        
        if name is 'group':
            group = value
        elif name is 'freq':
            freq = value
            regroup = True
        elif name is 'interval':
            interval = value
            regroup = True

        if regroup:
            groups = to_groups(peds, interval, freq)
            wGroup.max = len(groups) - 1
        
        pedestrians = to_pedestrians(peds, mapping, groups[group])
        od = create_od_matrix(pedestrians, source_target, precision=precision, cap=cap)
        
        origin = to_precision(od.sum(axis=0))
        destination = to_precision(od.sum(axis=1))
    
        ipysheet.renderer(code=generate_gradient_function(od.max()), name='gradient');
        ipysheet.renderer(code=generate_gradient_function(origin.max()), name='origin');
        ipysheet.renderer(code=generate_gradient_function(destination.max()), name='destination');
    
        ipysheet.cell_range(od, row_start=0, column_start=0, renderer='gradient')
        ipysheet.row(dim, origin, column_end=dim-1, renderer='origin')
        ipysheet.column(dim, destination, row_end=dim-1, renderer='destination')
        
        start = groups[group].datetime.iloc[0]
        end = groups[group].datetime.iloc[-1]
        wTime.value = "{:%d, %b %Y} from {:%I:%M:%S %p} to {:%I:%M:%S %p}".format(start, start, end)
        
            
    wFreq.observe(f, names='value')
    wInterval.observe(f, names='value')
    wGroup.observe(f, names='value')
    
    def save(button):
        data = []
        for g in groups:
            start = g.datetime.iloc[0]
            end = g.datetime.iloc[-1]
        
            # select mapping for pedestrians contained in given hour
            pedestrians = to_pedestrians(peds, mapping, g)
    
            # generate od matrix
            od = create_od_matrix(pedestrians, source_target, precision=precision, cap=cap)
    
            name = "{:%I:%M:%S %p} - {:%I:%M:%S %p}".format(start, end)
            data.append((name, len(pedestrians), od))
            
            #pd.DataFrame(od, columns=to_string(source_target)).to_excel(os.path.join(base, filename))
            
        write_xlsx(data, to_string(source_target), name="{:%Y%m%d-%H-%M-%S}-od-matrices-{}".format(datetime.now(), prefix))
    
            
    
    wSave.on_click(save)

    return widgets.VBox([wFreq, wInterval, wGroup, wTime, wSave, sheet])

In [4]:
def write_xlsx(groups, axis, name='od-matrices'):
    chars = 'ABCDEFGHIJKLMNOP'
    workbook = xlsxwriter.Workbook('{}.xlsx'.format(name))
    number_of_persons = workbook.add_worksheet('# of persons')
    od_matrices = workbook.add_worksheet('od matrices')
    
    format_center = workbook.add_format()
    format_center.set_align('center')
    
    format_col_sum = workbook.add_format()
    format_col_sum.set_align('center')
    format_col_sum.set_left(6)
    
    format_row_sum = workbook.add_format()
    format_row_sum.set_align('center')
    format_row_sum.set_top(6)
    
    format_row_index = workbook.add_format()
    format_row_index.set_align('center')
    format_row_index.set_right(6)
    
    number_of_persons.write_row(0, 0, ['Time', '# pedestrians'], format_center)
    
    # write header and freeze row
    od_matrices.write_row(0, 1, axis, format_center)
    od_matrices.freeze_panes(1, 0)
    
    n = len(axis)
    current = 1

    for j, group in tqdm(enumerate(groups)):
        time = group[0]
        persons = group[1]
        od = group[2]
        
        number_of_persons.write_row(j+1, 0, [time, persons], format_center)
        
        # write time
        od_matrices.merge_range(current, 0, current, n + 1, time, format_center)
        
        # write row index
        current = current + 1
        od_matrices.write_column(current, 0, axis, format_row_index)
        
        for row in range(n):
            od_matrices.write_row(current, 1, od[row], format_center)
            od_matrices.write_formula(current, n+1, '=SUM({}{}:{}{})'.format(chars[1], current+1, chars[n], current+1), format_col_sum)
            
            current = current + 1

            
        for col in range(n):
            i = col+1
            od_matrices.write_formula(current, i, '=SUM({}{}:{}{})'.format(chars[i], current-n+1, chars[i], current), format_row_sum)
        
        conditional_format = {
            'type': '3_color_scale', 
            'min_type': 'num', 
            'min_value': 0, 
            'mid_type': 'num', 
            'mid_value': 50, 
            'max_type': 'num', 
            'max_value': 100
        }
        
        # destination sum coloring
        od_matrices.conditional_format('{}{}:{}{}'.format(chars[1], current+1, chars[n], current+1), conditional_format)
        
        # origin sum coloring
        od_matrices.conditional_format('{}{}:{}{}'.format(chars[n+1], current-n + 1, chars[n+1], current), conditional_format)
        
        # distribution coloring
        od_matrices.conditional_format('{}{}:{}{}'.format(chars[1], current-n + 1, chars[n], current), conditional_format)
        
        current = current + 2
        
    workbook.close()
    print('done')
 

In [11]:
# base directory to use
#base = '../objectstream_20190905/'
base = '/media/veracrypt1/objectstream_20190905/'

frame = pd.read_csv(os.path.join(base, 'hours', 'hour-{}.csv'.format(12)), dtype={'pedestrianId': 'str'})
mapping = pd.read_csv(os.path.join(base, 'mapping.csv'), dtype={'pedestrianId': 'str'})

In [12]:
display(run(frame, mapping, prefix='05-12h'))